In [528]:
import pandas as pd
import plotly
from plotly import graph_objs as go
from plotly.graph_objs import *
from flask import Flask
import pandas as pd
import numpy as np
import os
import sqlite3
import copy
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import plotly.graph_objs as go
from matplotlib import ticker, cm
from matplotlib.pyplot import figure
import seaborn as sns
import plotly.graph_objects as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, plot_mpl
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [529]:
conn = sqlite3.connect("/Users/ankitkothari/Documents/COMPLETED_PROJECTS/H1B_data_analysis/us_h11b.db")

### Creating State Codes to Full Name Mapping

In [321]:

state_codes = pd.read_csv('/Users/ankitkothari/Downloads/csvData.csv')
state_codes
state_code2State = {v:k for k,v in zip(state_codes['State'],state_codes['Code'])}
state_code2State['VI']='Virgin Islands'
state_code2State['MP']='Northern Marianas'
state_code2State['PR']='Puerto Rico'
state_code2State['GU']='Guam'

### Total Approvals (New and Continuing) and Denials (New and Continuing) by Fiscal Year

In [97]:
h1b_query20 = ''' 
select  
Fiscal_Year,Employer,
SUM(Initial_Approvals)+ SUM(Continuing_Approvals) Approvals, 
SUM(Initial_Denials)+SUM(Continuing_Denials) AS Denials
from h1b
where Fiscal_Year !='2019'
group by Fiscal_Year, Employer
'''

In [98]:
df20 = pd.read_sql_query(h1b_query20, conn)
print(df20)

        Fiscal_Year                            Employer  Approvals  Denials
0              2009             & TV COMMUNICATIONS INC          1      0.0
1              2009     1 800 GIFT CERTIFICATES LLC DBA          3      0.0
2              2009     1 DEVELOPMENT CONSTRUCTION CORP          0      1.0
3              2009         1 I L INC DOING BUSINESS AS          1      0.0
4              2009                 1 RED W TRADING LLC          1      0.0
5              2009  1 RETAIL INVST SALES TEAM IN SO CA          1      0.0
6              2009              1 SOURCE SOLUTIONS LLC          1      1.0
7              2009          1 STOP TRANSLATION USA LLC          1      0.0
8              2009                  1 WAYSOLUTIONS INC          7      5.0
9              2009                    1&1 INTERNET INC          1      0.0
10             2009               1-800-FLOWERS.COM INC          5      0.0
11             2009             10 SQUARE SOLUTIONS INC          1      0.0
12          

In [99]:
df20.groupby('Fiscal_Year').sum()

,Approvals,Denials
Fiscal_Year,,
2009,227360,26051.0
2010,192369,12497.0
2011,269996,13123.0
2012,269804,11426.0
2013,295250,14134.0
2014,332788,16760.0
2015,288483,13071.0
2016,357181,23028.0
2017,373378,29855.0


In [530]:
def fiscal_plot(full_frame, Employer):
  try: 
        Employer=Employer.upper() 
  except:
        Employer=None
  if Employer is not None:
    full_frame=full_frame[full_frame['Employer']==Employer]
    full_frame=full_frame.groupby('Fiscal_Year').sum()
    df20=full_frame.reset_index()
    print(df20.head())
  else:
    full_frame=full_frame.groupby('Fiscal_Year').sum()
    df20=full_frame.reset_index()
    print(df20)
  fig = go.Figure()
  #fig.add_trace(go.Bar(x=df20.Fiscal_Year , y=df20.Approvals, mode='markers+lines', name='JOB TIME', line=dict(color='#e4bd0b', width=2)))
  fig.add_trace(go.Bar(x=[x for x in df20.Fiscal_Year] , y=df20.Approvals,marker_color='#00b8b8', name='Count of Approvals'))
  fig.add_trace(go.Scatter(x=[x for x in df20.Fiscal_Year], y=df20.Denials, mode='lines', name='Count of Denials', yaxis="y2", line=dict(color='#de3d83', width=4)))
  fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)
  fig.update_xaxes(
    dtick=1,showgrid=False
)
  fig.update_yaxes(
    showgrid=False
)
  fig.update_layout(title="Count of Approvals and Denials by Fiscal year", 
                 legend=dict(x=.03,y=0.98, traceorder='reversed', font_size=12), 
                 width=800,
                 height=400,
                 uniformtext_minsize=8,
                 uniformtext_mode='hide',
                 yaxis=dict(
        title="Count of Approvals (Bar)",
        titlefont=dict(
            color="#151515"
        ),
        tickfont=dict(
            color="#151515"
        )
    ),
    yaxis2=dict(
        title="Count of Denials (line)",
        titlefont=dict(
            color="#151515"
        ),
        tickfont=dict(
            color="#151515"
        ),
        anchor="free",
        side="right",
        overlaying="y",
        position=1
    ),)
  fig.update_layout(
    plot_bgcolor='#e0e5db'
)
  fig.add_shape(
        # Rectangle reference to the axes
            type="rect",
            xref="x",
            yref="paper",
            x0='2016',
            y0=-0.01,
            x1='2018',
            y1=1.1,
            line=dict(
                color="#e4bd0b",
                width=5,
            ),
        )
  return fig.show()

In [531]:
fiscal_plot(df20,Employer=None)

   Fiscal_Year  Approvals  Denials
0         2009     227360  26051.0
1         2010     192369  12497.0
2         2011     269996  13123.0
3         2012     269804  11426.0
4         2013     295250  14134.0
5         2014     332788  16760.0
6         2015     288483  13071.0
7         2016     357181  23028.0
8         2017     373378  29855.0
9         2018     334935  52738.0


### Approved H1B VISA Distribution by State

In [536]:
h1b_query13 = ''' 
with visas_by_state AS
(
select  
SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS sum_IA 
from h1b
where Fiscal_Year !='2019'
)
select  
State, SUM(Initial_Approvals)  + SUM(Continuing_Approvals) total_visa,
(CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL)) / (CAST(vs.sum_IA AS REAL))*100 AS percentage_total_visa
from h1b, visas_by_state vs
where Fiscal_Year !='2019'
group by State
order by 2 desc
;
'''

df13 = pd.read_sql_query(h1b_query13, conn)
df13['percentage_total_visa']=df13['percentage_total_visa'].apply(lambda x: round(x,2))
print(f' df13 {df13.head()}')



 df13   State  total_visa  percentage_total_visa
0    CA      496311                  16.87
1    TX      411275                  13.98
2    NJ      384639                  13.08
3    NY      228969                   7.78
4    IL      160014                   5.44


### Calculating the Number States whose Share is less than 1% in the total approved H1Bs

In [533]:
print(df13.info())
df13['category']= df13['percentage_total_visa'].apply(lambda x: "less Than 0" if x<1 else "Greater Than 0")
df13.groupby(['category']).count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
State                    55 non-null object
total_visa               55 non-null int64
percentage_total_visa    55 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.4+ KB
None


,State,total_visa,percentage_total_visa
category,,,
Greater Than 0,16,16,16
less Than 0,39,39,39


In [323]:
import plotly.graph_objects as go
y=[x  for  x in df13['percentage_total_visa'].iloc[::-1]]

fig = go.Figure(data=[go.Bar(
    y=[state_code2State[state] for state in df13['State'].iloc[::-1]],
    x=[x  for  x in df13['percentage_total_visa'].iloc[::-1]],
    width=.71,
    orientation='h',
    marker_color='#e4bd0b',
    text=y,
    textposition='outside',# marker color can be a single color value or an iterable
)])
fig.update_yaxes(tickangle = 360,tickfont=dict(family='Rockwell', color='#151515', size=12))
fig.update_traces(marker_line_width=.5, opacity=0.9)
fig.update_yaxes(dtick=1)
fig.update_xaxes(ticks="outside", tickwidth=3, tickcolor='#e0e5db', ticklen=12,showgrid=False)
fig.update_yaxes(ticks="outside", tickwidth=3, tickcolor='#e0e5db', ticklen=12)
fig.update_layout(title="Approved H1B Visa Distribution By State", 
                 legend=dict(x=.73,y=0.98, font_size=12), 
                 width=700,
                 height=900,
                 uniformtext_minsize=12,
                 xaxis=dict(title="% Share in Total Approved H1B Visas from 2009-2018",titlefont=dict(color="#151515"),
                            tickfont=dict(color="#151515")),)
fig.add_shape(
        # Rectangle reference to the axes
            type="rect",
            xref="paper",
            yref="paper",
            x0=0,
            y0=.91,
            x1=1.05,
            y1=1,
            line=dict(
                color="#de3d83",
                width=2,
            ),
        )
fig.add_annotation(
            x=.46,
            y=.90,
            text="57%",
            font=dict(
            family="Courier New, monospace",
            size=24,
            color="#00b8b8"
            ),)
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.add_annotation(
            x=.87,
            y=.90,
            text="of the Approved H1B Visas <br> comes from these 5 states",
            font=dict(
            family="Arial",
            size=12,
            color="#151515"
            ),)
fig.update_annotations(dict(
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=7,
            ax=50,
            ay=50
))

### The difference in Denial Rates pre and post-2016 by State


In [544]:
h1b_query8= ''' 
with h1b_table_by_state AS
(
select  
h1b.State,   
SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS approvals_pre_2016, 
SUM(h1b.Initial_Denials) + SUM(h1b.Continuing_Denials) AS denials_pre_2016,
(CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)) / (CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)+CAST(SUM(h1b.Initial_Approvals) AS REAL) + CAST(SUM(h1b.Continuing_Approvals) AS REAL))*100 AS denial_pre_2016,
h1b2.State,
h1b2.approvals_post_2016, 
h1b2.denials_post_2016,
h1b2.denial_post_2016

from h1b LEFT JOIN (
    select  
       State,  
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS approvals_post_2016, 
       SUM(Initial_Denials) + SUM(Continuing_Denials) AS denials_post_2016,
       (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)) / (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)+CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL))*100 AS denial_post_2016,
       Fiscal_Year
       from h1b
       where Fiscal_Year !='2019' and Fiscal_Year>2016
       group by State

) h1b2 ON h1b.State = h1b2.State
where h1b.Fiscal_Year !='2019' and h1b.Fiscal_Year<=2016
group by h1b.State
)
select 
hs.State,
hs.approvals_pre_2016 ,
hs.denials_pre_2016,
hs.denial_pre_2016 AS denial_rate_pre_2016,
hs.approvals_post_2016,
hs.denials_post_2016,
hs.denial_post_2016 AS denial_rate_post_2016,
hs.denial_post_2016 - hs.denial_pre_2016 AS delta
from h1b_table_by_state hs 
order by 2 desc
;
'''

df8 = pd.read_sql_query(h1b_query8, conn)
df8.set_index('State', inplace=True)
df8 = df8.applymap(lambda x: round(x,2))

df8=df8.reset_index()
df8=df8.sort_values(by='delta',ascending=False)
df8=df8.iloc[::-1]

In [541]:
import plotly.graph_objects as go

colors  = ['#e4bd0b' if x < 0 else '#de3d83' for x in df8['delta']]
fig = go.Figure(data=[go.Bar(
    y=[state_code2State[x] for x in df8['State']],
    x=[y for  y in df8['delta']],
    width=.51,
    orientation='h',
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.update_yaxes(
        tickangle = 360,
        tickfont=dict(family='Rockwell', color='#2677bb', size=8))
fig.update_yaxes(tickangle = 360,tickfont=dict(family='Rockwell', color='#151515', size=12))
fig.update_traces(marker_line_width=.5, opacity=0.9)
fig.update_yaxes(dtick=1)
fig.update_xaxes(dtick=1,ticks="outside", tickwidth=3, tickcolor='#e0e5db', ticklen=12,showgrid=False)
fig.update_yaxes(ticks="outside", tickwidth=3, tickcolor='#e0e5db', ticklen=12)
fig.update_traces(marker_line_width=.5, opacity=0.9)
fig.update_layout(title="Difference in Denial Rates pre and post 2016", 
                 legend=dict(x=.73,y=0.98, traceorder='reversed', font_size=12), 
                 width=605,
                 height=900,
                 uniformtext_minsize=12,
                 uniformtext_mode='hide',
                 xaxis=dict(
        title="Denial Rate %",
        titlefont=dict(
            color="#151515"
        ),
        tickfont=dict(
            color="#151515"
        )
    ),
    )

### How is the use of H1B Visas distributed among Employers?

In [545]:
h1b_query21= ''' 
with h1b_table_by_state AS
(
select  
h1b.Employer,   
SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS approvals_pre_2016, 
SUM(h1b.Initial_Denials) + SUM(h1b.Continuing_Denials) AS denials_pre_2016,
(CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)) / (CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)+CAST(SUM(h1b.Initial_Approvals) AS REAL) + CAST(SUM(h1b.Continuing_Approvals) AS REAL))*100 AS denial_pre_2016,
h1b2.Employer,
h1b2.approvals_post_2016, 
h1b2.denials_post_2016,
h1b2.denial_post_2016

from h1b LEFT JOIN (
    select  
       Employer,  
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS approvals_post_2016, 
       SUM(Initial_Denials) + SUM(Continuing_Denials) AS denials_post_2016,
       (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)) / (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)+CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL))*100 AS denial_post_2016,
       Fiscal_Year
       from h1b
       where Fiscal_Year !='2019' and Fiscal_Year>2016
       group by Employer

) h1b2 ON h1b.Employer = h1b2.Employer
where h1b.Fiscal_Year !='2019' and h1b.Fiscal_Year<=2016
group by h1b.Employer
having approvals_pre_2016 > 2000 and h1b2.approvals_post_2016>500
)
select 
hs.Employer,
CASE WHEN hs.Employer IN ("WAL-MART","APPLE","FACEBOOK","AMAZON","MICROSOFT","GOOGLE") THEN "US COMPANIES" ELSE "INDIAN COMPANIES" END AS COMPANY_ORIGIN,
hs.denial_pre_2016 AS denial_rate_pre_2016,
hs.denial_post_2016 AS denial_rate_post_2016,
hs.denial_post_2016 - hs.denial_pre_2016 AS delta_denial_rates_pre_post2016
from h1b_table_by_state hs 
where hs.Employer IN ("WAL-MART","APPLE","FACEBOOK","AMAZON","MICROSOFT","GOOGLE", "TATA", "ACCENTURE", "WIPRO","CAPGEMINI","MINDTREE")
order by 4 desc
limit 15
;
'''

df21 = pd.read_sql_query(h1b_query21, conn)
df21[['denial_rate_pre_2016','denial_rate_post_2016','delta_denial_rates_pre_post2016']]=df21[['denial_rate_pre_2016','denial_rate_post_2016','delta_denial_rates_pre_post2016']].apply(lambda x: round(x,2))
df21=df21.sort_values(by=['COMPANY_ORIGIN'])

In [549]:
df21

,Employer,COMPANY_ORIGIN,denial_rate_pre_2016,denial_rate_post_2016,delta_denial_rates_pre_post2016
10,FACEBOOK,US COMPANIES,0.46,0.73,0.27
9,APPLE,US COMPANIES,0.55,1.22,0.66
8,GOOGLE,US COMPANIES,0.40,1.35,0.95
7,MICROSOFT,US COMPANIES,1.03,1.72,0.69
6,AMAZON,US COMPANIES,0.85,1.84,0.99
5,TATA,INDIAN COMPANIES,5.16,4.52,-0.63
4,WAL-MART,US COMPANIES,1.26,6.20,4.94
3,ACCENTURE,INDIAN COMPANIES,2.28,8.93,6.65
2,WIPRO,INDIAN COMPANIES,4.67,9.66,4.99
1,MINDTREE,INDIAN COMPANIES,3.12,10.40,7.29


In [548]:
df21.groupby('COMPANY_ORIGIN').mean()['delta_denial_rates_pre_post2016']

COMPANY_ORIGIN
INDIAN COMPANIES    7.782000
US COMPANIES        1.416667
Name: delta_denial_rates_pre_post2016, dtype: float64

In [546]:
df21=df21.sort_values(by=['denial_rate_post_2016'], ascending=True)
def denial_plot(full_frame):

  fig = go.Figure()
  y1=[str(x)  for  x in df21['denial_rate_pre_2016']]
  y2=[str(x)  for  x in df21['denial_rate_post_2016']]
  #fig.add_trace(go.Bar(x=df20.Fiscal_Year , y=df20.Approvals, mode='markers+lines', name='JOB TIME', line=dict(color='#e4bd0b', width=2)))
  fig.add_trace(go.Bar(y=[x for x in df21.Employer] , x=df21.denial_rate_pre_2016,marker_color='#00b8b8',orientation='h', name='Denial Rate Pre 2016', text=y1,
    textposition='outside'))
  fig.add_trace(go.Bar(y=[x for x in df21.Employer] , x=df21.denial_rate_post_2016,marker_color='#e4bd0b',orientation='h', name='Denial Rate Post 2016',text=y2,
    textposition='outside'))
  #fig.add_trace(go.Scatter(x=[x for x in df20.Fiscal_Year], y=df20.Denials, mode='lines', name='Count of Denials', yaxis="y2", line=dict(color='#bfbabe', width=4)))
  fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)
  fig.update_xaxes(
        tickangle = 0,
        tickfont=dict(family='Rockwell', color='#151515', size=16))

  fig.update_xaxes(
    dtick=2, showgrid=False
)
  fig.update_yaxes(
    dtick=1,showgrid=False
)
  fig.update_yaxes(ticks="outside", tickwidth=3, tickcolor='#e0e5db', ticklen=12)
  fig.update_layout(title="Denial Rates Indian IT Companies Vs US TechGiants", 
                 legend=dict(x=.73,y=0.78, traceorder='reversed', font_size=12), 
                 width=600,
                 height=600,
                 uniformtext_minsize=12,
                 yaxis=dict(
        title="",
        titlefont=dict(
            color="#151515"
        ),
        tickfont=dict(
            color="#151515"
        )
    ),
                  xaxis=dict(title="% Denial Rate",titlefont=dict(color="#151515"),
                            tickfont=dict(color="#151515")),)


  return fig.show()

In [547]:
denial_plot(df21)

### Distinct Employers Using the H1B program by State

In [557]:
h1b_query35 = ''' 
select  
State, count(DISTINCT Employer) Employer
from h1b
where Fiscal_Year !='2019'
group by 1
;'''


df35 = pd.read_sql_query(h1b_query35, conn)
df35=df35.sort_values(by='Employer', ascending=False)
df35=df35.reset_index()
df35['cumsum']=df35['Employer'].cumsum()
df35.head()


,index,State,Employer,cumsum
0,4,CA,53969,53969
1,36,NY,37496,91465
2,46,TX,24124,115589
3,9,FL,19501,135090
4,33,NJ,14957,150047


In [558]:
import plotly.graph_objects as go
colors  = ['#de3d83' if x < 1000 else '#e4bd0b' if x<10000 else "#00b8b8" for x in df35['Employer']]
fig = go.Figure(data=go.Choropleth(
    locations=df35['State'], # Spatial coordinates
    z = df35['Employer'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = colors,
    colorbar_title = "Distinct Employers",
))

fig.update_layout(
    title_text = 'Distinct Employers Applying for H1B in US By States',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

### Total Distinct Employers in the US who have used H1B between 2009-2018

In [534]:
h1b_query36 = ''' 
select  
count(DISTINCT Employer) Employer
from h1b
where Fiscal_Year !='2019'
;'''


df36 = pd.read_sql_query(h1b_query36, conn)
df36

,Employer
0,265149


### How is the use of H1B Visas distributed among Employers?

In [288]:
h1b_query14 = ''' 
with visas_by_Employer AS
(
select  
SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS sum_IA 
from h1b
where Fiscal_Year !='2019'
)
select  
Employer,
(CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL)) / (CAST(vs.sum_IA AS REAL))*100 AS percent_total_H1B 
from h1b, visas_by_Employer vs
where Fiscal_Year !='2019'
group by Employer
order by 2 desc
limit 20
;
'''

df14 = pd.read_sql_query(h1b_query14, conn)
print(f' df14 {df14}')
print(f' df14 {df14.sum()[1]}')
df14['percent_total_H1B']=df14['percent_total_H1B'].apply(lambda x:round(x,2))

 df14                       Employer  percent_total_H1B
0                    COGNIZANT           4.887569
1                      INFOSYS           3.330258
2                         TATA           2.564470
3                        WIPRO           2.040153
4                    ACCENTURE           1.249004
5                    MICROSOFT           1.229762
6                     DELOITTE           1.184480
7                          IBM           1.034593
8                          HCL           0.874541
9                       AMAZON           0.809745
10                      LARSEN           0.692052
11                       INTEL           0.644865
12                      GOOGLE           0.634259
13  TECH MAHINDRA AMERICAS INC           0.602575
14                      SYNTEL           0.601997
15                   CAPGEMINI           0.557224
16                 ERNST&YOUNG           0.521903
17                    QUALCOMM           0.481720
18                      ORACLE           0.4

In [295]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Bar(
    y=[x for x in df14['Employer'].iloc[::-1]],
    x=[y for  y in df14['percent_total_H1B'].iloc[::-1]],
    width=.51,
    orientation='h',
    marker_color='#00b8b8',
    text=[y for  y in df14['percent_total_H1B'].iloc[::-1]],
    textposition='outside',# marker color can be a single color value or an iterable
)])
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.update_yaxes(
        tickangle = 360,
        tickfont=dict(family='Rockwell', color='#2677bb', size=8))
fig.update_yaxes(tickangle = 360,tickfont=dict(family='Rockwell', color='#151515', size=12))
fig.update_traces(marker_line_width=.5, opacity=0.9)
fig.update_yaxes(dtick=1)
fig.update_xaxes(dtick=1,ticks="outside", tickwidth=3, tickcolor='#e0e5db', ticklen=12,showgrid=False)
fig.update_yaxes(ticks="outside", tickwidth=3, tickcolor='#e0e5db', ticklen=12)
fig.update_traces(marker_line_width=.5, opacity=0.9)
fig.update_layout(title="Top 20 Companies Using H1B VISAS", 
                 legend=dict(x=.73,y=0.98, traceorder='reversed', font_size=12), 
                 width=500,
                 height=600,
                 uniformtext_minsize=14,
                 xaxis=dict(
        title="% Share in Total Approved Applications",
        titlefont=dict(
            color="#151515"
        ),
        tickfont=dict(
            color="#151515"
        )
    ),
    )

### How does the Employer State relationship look like?


In [550]:
h1b_query9= '''
with h1b_table_by_state AS
(
select  
h1b.State AS State,
h1b.Employer AS Employer,
SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS total_visas_State, 
Max(h1b2.approvals_post_2016) AS max_visa_in_state
from h1b LEFT JOIN (
    select  
       State, Employer,
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS approvals_post_2016
       from h1b
       where Fiscal_Year !='2019'
       group by State,Employer
       order by 3 desc

) h1b2 ON h1b.State = h1b2.State and h1b.Employer = h1b2.Employer 
where h1b.Fiscal_Year !='2019'
group by h1b.State
order by 4 desc
)
select
htbs.State,
htbs.Employer,
htbs.total_visas_State,
htbs.max_visa_in_state,
(CAST(htbs.max_visa_in_state AS REAL)/ CAST(htbs.total_visas_State AS REAL))*100 AS percent_visa
from h1b_table_by_state htbs 
order by 5 desc;
'''

df9 = pd.read_sql_query(h1b_query9, conn)
#print(f' df 9 \n {df9}')
df9['percent_visa']=df9['percent_visa'].apply(lambda x: round(x,0))
df9.head()


,State,Employer,total_visas_State,max_visa_in_state,percent_visa
0,MD,TATA,121999,74504,61.0
1,VT,ITECH US INC,2994,1677,56.0
2,NC,IBM,63728,30357,48.0
3,ID,MICRON TECHNOLOGY INC,3914,1791,46.0
4,AR,WAL-MART,10463,4120,39.0


In [494]:
df9a=df9[df9['percent_visa']>10]
df9a['State']=df9a['State'].map(state_code2State)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [551]:
df9a=df9[df9['percent_visa']>10]
df9a['State']=df9a['State'].map(state_code2State)
df9d=df9a.copy(deep=True)
df9d['Employer']=df9d['Employer'].str.replace('MICRON TECHNOLOGY INC','MICRON <br> TECHNOLOGY INC')
df9d['Employer']=df9d['Employer'].str.replace('Alaska-UNIV OF ALASKA FAIRBANKS','Alaska-UNIV OF <br> ALASKA FAIRBANKS')
df9d['Employer']=df9d['Employer'].str.replace('PRICEWATERHOUSECOOPERS','PWC')
df9d['Employer']=df9d['Employer'].str.replace('THE UNIV OF ALABAMA AT BIRMINGHAM','THE UNIV OF ALABAMA <br> AT BIRMINGHAM')
df9d['Employer']=df9d['Employer'].str.replace('V-SOFT CONSULTING GROUP INC','V-SOFT CONSULTING')
df9d['Employer']=df9d['Employer'].str.replace('UNIV OF ALASKA FAIRBANKS','UNIV OF ALASKA <br> FAIRBANKS')
df9d['Employer']=df9d['Employer'].str.replace('SOUTH DAKOTA STATE UNIVERSITY','SOUTH DAKOTA <br> STATE UNIVERSITY')
df9d['Employer']=df9d['Employer'].str.replace('MONTANA STATE UNIVERSITY','MONTANA STATE <br> UNIVERSITY')
df9d['Employer']=df9d['Employer'].str.replace('LOS ALAMOS NATIONAL SECURITY LLC','LOS ALAMOS NATIONAL <br> SECURITY LLC')

df9d['Employer']=["<br>".join([x,str(z)]) for x,z in zip(df9d['Employer'],df9d['percent_visa'])]
df9d['Employer']=[" ".join([str(z),'%']) for z in df9d['Employer']]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [552]:
df9d['ids']= df9d['State'].to_list()
df9d['parents']=""
df9d

,State,Employer,total_visas_State,max_visa_in_state,percent_visa,ids,parents
0,Maryland,TATA<br>61.0 %,121999,74504,61.0,Maryland,
1,Vermont,ITECH US INC<br>56.0 %,2994,1677,56.0,Vermont,
2,North Carolina,IBM<br>48.0 %,63728,30357,48.0,North Carolina,
3,Idaho,MICRON <br> TECHNOLOGY INC<br>46.0 %,3914,1791,46.0,Idaho,
4,Arkansas,WAL-MART<br>39.0 %,10463,4120,39.0,Arkansas,
5,Washington,MICROSOFT<br>38.0 %,95411,36151,38.0,Washington,
6,Tennessee,CUMMINS<br>27.0 %,24813,6773,27.0,Tennessee,
7,Pennsylvania,DELOITTE<br>26.0 %,109917,28968,26.0,Pennsylvania,
8,Rhode Island,CVS/PHARMACY<br>24.0 %,7072,1679,24.0,Rhode Island,
9,Texas,INFOSYS<br>24.0 %,411275,97180,24.0,Texas,


In [553]:
df9e=df9d.copy(deep=True)
df9e['ids']=["-".join([x,y]) for x,y in zip(df9e['State'],df9e['Employer'])]
df9e['parents']=df9e.State.to_list()
df9e.head()

,State,Employer,total_visas_State,max_visa_in_state,percent_visa,ids,parents
0,Maryland,TATA<br>61.0 %,121999,74504,61.0,Maryland-TATA<br>61.0 %,Maryland
1,Vermont,ITECH US INC<br>56.0 %,2994,1677,56.0,Vermont-ITECH US INC<br>56.0 %,Vermont
2,North Carolina,IBM<br>48.0 %,63728,30357,48.0,North Carolina-IBM<br>48.0 %,North Carolina
3,Idaho,MICRON <br> TECHNOLOGY INC<br>46.0 %,3914,1791,46.0,Idaho-MICRON <br> TECHNOLOGY INC<br>46.0 %,Idaho
4,Arkansas,WAL-MART<br>39.0 %,10463,4120,39.0,Arkansas-WAL-MART<br>39.0 %,Arkansas


In [554]:
df9f= pd.concat([df9d,df9e], axis=0)
df9f['colors']  = ['#00b8b8' if x < 15 else '#e4bd0b' if x<50 else "#de3d83" for x in df9f['percent_visa']]
df9f.head()

,State,Employer,total_visas_State,max_visa_in_state,percent_visa,ids,parents,colors
0,Maryland,TATA<br>61.0 %,121999,74504,61.0,Maryland,,#de3d83
1,Vermont,ITECH US INC<br>56.0 %,2994,1677,56.0,Vermont,,#de3d83
2,North Carolina,IBM<br>48.0 %,63728,30357,48.0,North Carolina,,#e4bd0b
3,Idaho,MICRON <br> TECHNOLOGY INC<br>46.0 %,3914,1791,46.0,Idaho,,#e4bd0b
4,Arkansas,WAL-MART<br>39.0 %,10463,4120,39.0,Arkansas,,#e4bd0b


In [525]:
labelsa= [state for state in df9f['State'].unique()]
labelsb = [x for x in df9f['Employer'].unique()]
labels= labelsa+labelsb
parents = [x for x in df9f['parents']]
print(f' labels {labels}')
print(f' parents {parents}')

 labels ['Maryland', 'Vermont', 'North Carolina', 'Idaho', 'Arkansas', 'Washington', 'Tennessee', 'Pennsylvania', 'Rhode Island', 'Texas', 'Illinois', 'Wyoming', 'Arizona', 'Alaska', 'Kentucky', 'Michigan', 'Montana', 'New Jersey', 'West Virginia', 'Florida', 'Nebraska', 'Virgin Islands', 'Hawaii', 'South Dakota', 'Iowa', 'Alabama', 'New Mexico', 'Maine', 'TATA<br>61.0 %', 'ITECH US INC<br>56.0 %', 'IBM<br>48.0 %', 'MICRON <br> TECHNOLOGY INC<br>46.0 %', 'WAL-MART<br>39.0 %', 'MICROSOFT<br>38.0 %', 'CUMMINS<br>27.0 %', 'DELOITTE<br>26.0 %', 'CVS/PHARMACY<br>24.0 %', 'INFOSYS<br>24.0 %', 'ACCENTURE<br>23.0 %', 'UNIV OF WYOMING<br>22.0 %', 'INTEL<br>19.0 %', 'UNIV OF ALASKA <br> FAIRBANKS<br>19.0 %', 'V-SOFT CONSULTING<br>18.0 %', 'SYNTEL<br>18.0 %', 'MONTANA STATE <br> UNIVERSITY<br>16.0 %', 'WIPRO<br>16.0 %', 'WEST VIRGINIA UNIV<br>15.0 %', 'PWC<br>15.0 %', 'PROKARMA INC<br>15.0 %', 'U S VIRGIN ISLANDS DEPT OF ED<br>14.0 %', 'UNIVERSITY OF HAWAII<br>14.0 %', 'SOUTH DAKOTA <br> STATE UN

In [555]:
ids=[x for x in df9f['ids']]
ids[0:5]

['Maryland', 'Vermont', 'North Carolina', 'Idaho', 'Arkansas']

In [527]:
import plotly.graph_objects as go

fig =go.Figure(go.Sunburst(
 ids=[x for x in df9f['ids']],
  labels= labels,
  parents=parents,
  marker=dict(
        colors=df9f['colors']
    )
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()